# 모듈 불러오기

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

# data전처리

### song 전처리

In [3]:
# 노래 데이터(다운로드 횟수, 재생 횟수)
song_data = pd.read_csv('data\song.csv', encoding='utf-8')
# 컬럼명 수정
song_data.rename(columns={'created_at':'downloaded_at'}, inplace=True)
song_data.rename(columns={'length':'length(sec)'}, inplace=True) 
# 데이트타임 타입으로 변경하고 연월일 추출
song_data['downloaded_at'] = pd.to_datetime(song_data['downloaded_at'])
song_data['downloaded_at'] = song_data['downloaded_at'].dt.strftime("%Y-%m-%d")
song_data['downloaded_at'] = pd.to_datetime(song_data['downloaded_at'])
song_data

,id,is_free,download_count,play_count,downloaded_at,is_visible,grade,bpm,length(sec)
0,2,True,6,49,2018-05-12,False,1.3.F,116.0,227
1,3,True,3,110,2018-05-12,False,1.3.F,175.0,244
2,4,True,1,52,2018-05-12,False,1.3.F,160.0,347
3,5,True,1,32,2018-05-12,False,1.3.F,116.0,230
4,6,True,2,22,2018-05-12,False,1.3.F,173.0,318
...,...,...,...,...,...,...,...,...,...
6137,9462,False,4,20,2022-10-11,True,1.1.A,150.0,125
6138,9463,False,0,0,2022-10-11,True,1.1.A,110.0,284
6139,9464,False,1,13,2022-10-11,True,1.1.A,110.0,142
6140,9465,False,0,0,2022-10-11,True,1.1.A,128.0,242


In [4]:
# song 데이터 결측치 확인
song_data[song_data['bpm'].isnull()]

,id,is_free,download_count,play_count,downloaded_at,is_visible,grade,bpm,length(sec)
643,3479,False,0,0,2019-04-11,False,1.1.A,NaN,101
645,3481,False,0,0,2019-04-11,False,1.1.A,NaN,77
701,3537,False,0,0,2019-04-11,False,1.1.A,NaN,96
705,3541,False,0,0,2019-04-11,False,1.1.A,NaN,108
801,3638,False,0,10,2019-04-18,True,1.4.D,NaN,120
844,3729,False,0,0,2019-04-11,False,1.1.A,NaN,120
936,3821,False,0,0,2019-04-11,False,1.1.A,NaN,115
1013,3900,False,0,0,2019-04-11,False,1.1.A,NaN,96
1014,3901,False,0,0,2019-04-11,False,1.1.A,NaN,115


In [5]:
# song데이터 결측치 처리
song_data = song_data.dropna()
song_data.head(10)

,id,is_free,download_count,play_count,downloaded_at,is_visible,grade,bpm,length(sec)
0,2,True,6,49,2018-05-12,False,1.3.F,116.0,227
1,3,True,3,110,2018-05-12,False,1.3.F,175.0,244
2,4,True,1,52,2018-05-12,False,1.3.F,160.0,347
3,5,True,1,32,2018-05-12,False,1.3.F,116.0,230
4,6,True,2,22,2018-05-12,False,1.3.F,173.0,318
5,9,True,1,12,2018-05-19,False,1.3.F,166.0,199
6,10,True,7,176,2018-05-19,False,1.3.F,120.0,169
7,11,True,2,27,2018-05-19,False,1.3.F,188.0,177
8,12,True,3,41,2018-05-19,False,1.3.F,120.0,292
9,13,True,8,187,2018-05-19,False,1.3.F,179.0,232


In [6]:
song_data.isnull().sum()

id                0
is_free           0
download_count    0
play_count        0
downloaded_at     0
is_visible        0
grade             0
bpm               0
length(sec)       0
dtype: int64

### unsubscribe_request 전처리

In [7]:
# 구독 취소 요청 데이터
unsub_data = pd.read_csv('data/unsubscribe_request.csv', encoding= 'utf-8')

# 컬럼명 수정
unsub_data.rename(columns={'time':'unsubscribed_at'}, inplace=True)
# 데이트타임 타입으로 변경하고 연월일 추출
unsub_data['unsubscribed_at'] = pd.to_datetime(unsub_data['unsubscribed_at'])
unsub_data['unsubscribed_at'] = unsub_data['unsubscribed_at'].dt.strftime("%Y-%m-%d")
unsub_data['unsubscribed_at'] = pd.to_datetime(unsub_data['unsubscribed_at'])


### interest_pricing 전처리

In [8]:
pricing =pd.read_csv("data/interest_pricing.csv")

In [9]:
pricing.isnull().sum()

id         0
user_id    0
name       0
time       0
dtype: int64

In [10]:
# Datatime 변경
pricing['time'] = pd.to_datetime(pricing['time'])
# 연월일 추출
pricing['time'] =pricing['time'].dt.strftime('%Y-%m-%d')
pricing['time'] = pd.to_datetime(pricing['time'])

pricing.head(3)

,id,user_id,name,time
0,6168,89167,Viewed Subscription,2022-02-04
1,6086,51726,Viewed Subscription,2022-02-04
2,6170,89167,Viewed Subscription,2022-02-04


### tag 전처리

In [11]:
tag = pd.read_csv("data/tag.csv")
tag.isnull().sum()

id              0
name          659
model_id        0
model_type      0
dtype: int64

In [12]:
# 'name' null 행 삭제
tag[tag['name'].isnull()]

,id,name,model_id,model_type
606,36754,NaN,997,Music::SoundEffect
825,47447,NaN,1370,Music::SoundEffect
8785,65261,NaN,495,Music::SoundEffect
8834,65795,NaN,361,Music::SoundEffect
9383,70786,NaN,2352,Music::SoundEffect
...,...,...,...,...
154051,306442,NaN,8317,Music::Song
154064,306455,NaN,8318,Music::Song
154077,306468,NaN,8319,Music::Song
154090,306481,NaN,8320,Music::Song


In [13]:
tag=tag.dropna(axis=0)

In [15]:
tag.isnull().sum()

id            0
name          0
model_id      0
model_type    0
dtype: int64

### user 전처리(범주형 데이터처리)

In [16]:
user = pd.read_csv("data/user.csv")
user.isnull().sum()

id                     0
sign_in_count          0
last_sign_in_at     1170
user_type              0
created_at             0
favorite_genre     76217
dtype: int64

In [19]:
# 데이트타임 타입으로 변경하고 연월일 추출 & # 컬럼명 수정
user['created_at'] = pd.to_datetime(user['created_at'])
user['created_at'] = user['created_at'].dt.strftime('%Y-%m-%d')
user['created_at'] = pd.to_datetime(user['created_at'])

user['last_sign_in_at'] = user['last_sign_in_at'].fillna(user['created_at'])
user['last_sign_in_at'] = pd.to_datetime(user['last_sign_in_at'])
user['last_sign_in_at']=user['last_sign_in_at'].dt.strftime('%Y-%m-%d')
user['last_sign_in_at'] = pd.to_datetime(user['last_sign_in_at'])
user.rename(columns={'last_sign_in_at':'last_login_at'},inplace=True)

In [24]:
# 범주형 데이터 전처리
# favorite_genre null값 채우기
user['favorite_genre']=user['favorite_genre'].fillna('[]')

# 장르들을 리스트에 담기
items = ['kids', 'vlog', 'story', 'beauty', 'review', 'talk', 'game', 'pet', 'eating', 'drone']

# favorite_genre 칼럼 만들기
for i in items:
  user[i] = user['favorite_genre'].str.contains(i, na=False)
user

,user_id,sign_in_count,last_login_at,user_type,created_at,favorite_genre,kids,vlog,story,beauty,review,talk,game,pet,eating,drone
0,1,7201,2022-10-12,unknown,2018-05-11,"[""vlog""]",False,True,False,False,False,False,False,False,False,False
1,12,49,2021-10-08,individual,2018-05-30,[],False,False,False,False,False,False,False,False,False,False
2,29,23,2022-07-27,student,2018-08-02,[],False,False,False,False,False,False,False,False,False,False
3,30,241,2022-08-24,unknown,2018-08-06,[],False,False,False,False,False,False,False,False,False,False
4,31,0,2018-08-07,unknown,2018-08-07,[],False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102789,118123,1,2022-10-12,individual,2022-10-12,[],False,False,False,False,False,False,False,False,False,False
102790,118124,1,2022-10-12,unknown,2022-10-12,[],False,False,False,False,False,False,False,False,False,False
102791,118125,1,2022-10-12,corporate,2022-10-12,[],False,False,False,False,False,False,False,False,False,False
102792,118126,1,2022-10-12,individual,2022-10-12,[],False,False,False,False,False,False,False,False,False,False


In [25]:
for i in items:
  user[i] = user[i].astype(int)

In [26]:
user.rename(columns={'id':'user_id'}, inplace = True)
user

,user_id,sign_in_count,last_login_at,user_type,created_at,favorite_genre,kids,vlog,story,beauty,review,talk,game,pet,eating,drone
0,1,7201,2022-10-12,unknown,2018-05-11,"[""vlog""]",0,1,0,0,0,0,0,0,0,0
1,12,49,2021-10-08,individual,2018-05-30,[],0,0,0,0,0,0,0,0,0,0
2,29,23,2022-07-27,student,2018-08-02,[],0,0,0,0,0,0,0,0,0,0
3,30,241,2022-08-24,unknown,2018-08-06,[],0,0,0,0,0,0,0,0,0,0
4,31,0,2018-08-07,unknown,2018-08-07,[],0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102789,118123,1,2022-10-12,individual,2022-10-12,[],0,0,0,0,0,0,0,0,0,0
102790,118124,1,2022-10-12,unknown,2022-10-12,[],0,0,0,0,0,0,0,0,0,0
102791,118125,1,2022-10-12,corporate,2022-10-12,[],0,0,0,0,0,0,0,0,0,0
102792,118126,1,2022-10-12,individual,2022-10-12,[],0,0,0,0,0,0,0,0,0,0


### subscription_log 전처리(이상치 삭제)

In [27]:
sub = pd.read_csv("data/subscription_log.csv")
sub.isnull().sum()

id                     0
user_id                0
end_date               0
price                205
created_at             0
refund_id          30314
point               2110
target_customer        1
dtype: int64

In [28]:
# price null값 처리
sub=sub.dropna(subset=['price'])
sub['price'].isnull().sum()

0

In [29]:
# 이상치 데이터 삭제
del sub['point']

In [30]:
sub[sub['end_date']>'9990']

,id,user_id,end_date,price,created_at,refund_id,target_customer
27839,296239,108213,9999-12-31 00:00:00,0.0,2022-08-05 08:56:24.381447,NaN,smallBiz
28001,297370,108213,9999-12-31 00:00:00,0.0,2022-08-08 13:10:53.116591,NaN,smallBiz


In [31]:
notdate = sub[sub['end_date'].str.contains('9999')].index
sub.drop(notdate, inplace=True)

In [32]:
(sub['end_date']>'9990').sum()

0

In [33]:
sub['end_date']=pd.to_datetime(sub['end_date'])
sub['end_date'] = sub['end_date'].dt.strftime('%Y-%m-%d')
sub['end_date']=pd.to_datetime(sub['end_date'])
sub['created_at']=pd.to_datetime(sub['created_at'])
sub['created_at'] = sub['created_at'].dt.strftime('%Y-%m-%d')
sub['created_at']=pd.to_datetime(sub['created_at'])

In [34]:
sub=sub.dropna(subset=['target_customer'])
sub.isnull().sum()

id                     0
user_id                0
end_date               0
price                  0
created_at             0
refund_id          30107
target_customer        0
dtype: int64

In [35]:
sub

,id,user_id,end_date,price,created_at,refund_id,target_customer
0,237,6482,2019-08-12,100.0,2019-07-12,NaN,personal
1,239,6574,2019-08-12,100.0,2019-07-12,NaN,personal
2,252,6704,2019-08-16,100.0,2019-07-16,NaN,personal
3,258,4709,2019-08-16,9900.0,2019-07-17,NaN,personal
4,259,6748,2019-08-17,100.0,2019-07-17,NaN,personal
...,...,...,...,...,...,...,...
31056,330385,118082,2022-10-19,0.0,2022-10-12,NaN,smallBiz
31057,330421,117845,2023-10-12,118800.0,2022-10-12,NaN,personal
31058,330467,118102,2022-10-19,0.0,2022-10-12,NaN,personal
31059,330471,84676,2022-10-19,0.0,2022-10-12,NaN,personal


In [36]:
# refund_id 변경 refund 유무 만 확인
sub.loc[sub['refund_id'] > 1,'refund_id']=1
sub=sub.fillna('0')
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30853 entries, 0 to 31060
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               30853 non-null  int64         
 1   user_id          30853 non-null  int64         
 2   end_date         30853 non-null  datetime64[ns]
 3   price            30853 non-null  float64       
 4   created_at       30853 non-null  datetime64[ns]
 5   refund_id        30853 non-null  object        
 6   target_customer  30853 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 1.9+ MB


In [37]:
sub.rename(columns={'refund_id':'refund'},inplace=True)
sub

,id,user_id,end_date,price,created_at,refund,target_customer
0,237,6482,2019-08-12,100.0,2019-07-12,0,personal
1,239,6574,2019-08-12,100.0,2019-07-12,0,personal
2,252,6704,2019-08-16,100.0,2019-07-16,0,personal
3,258,4709,2019-08-16,9900.0,2019-07-17,0,personal
4,259,6748,2019-08-17,100.0,2019-07-17,0,personal
...,...,...,...,...,...,...,...
31056,330385,118082,2022-10-19,0.0,2022-10-12,0,smallBiz
31057,330421,117845,2023-10-12,118800.0,2022-10-12,0,personal
31058,330467,118102,2022-10-19,0.0,2022-10-12,0,personal
31059,330471,84676,2022-10-19,0.0,2022-10-12,0,personal


In [38]:
# 이상데이터 user_id 1~40번 삭제
sub[sub['user_id'] <= 40]
notid = sub[sub['user_id'] <= 40].index
sub.drop(notid, inplace=True)
(sub['user_id']<=40).sum()

0

### download 전처리

In [41]:
df_down=pd.read_csv('data\download.csv')
df_down.head()

,user_id,model_id,model_type,created_at
0,13649,6114,Music::Song,2021-03-16 00:02:38.546267
1,55815,2574,Music::SoundEffect,2021-03-16 00:04:19.270351
2,55815,2562,Music::SoundEffect,2021-03-16 00:04:45.522705
3,57150,3512,Music::SoundEffect,2021-03-16 00:07:41.811291
4,55815,3527,Music::SoundEffect,2021-03-16 00:08:42.888535


In [43]:
# del df_down['Unnamed: 0']
df_down['model_type'].unique()

array(['Music::Song', 'Music::SoundEffect', 'Scene::Image',
       'Classify::Package', 'Scene::VideoTemplate'], dtype=object)

In [47]:
df_down['model_type']=df_down['model_type'].str.replace('Music::','').replace('Scene::Image','etc').replace('Classify::Package','etc').replace('Scene::VideoTemplate','etc')
df_down.rename(columns={'created_at':'downloaded_at'},inplace=True)
df_down['downloaded_at'] = pd.to_datetime(df_down['downloaded_at'])
df_down['downloaded_at'] = df_down['downloaded_at'].dt.strftime('%Y-%m-%d')
df_down

,user_id,model_id,model_type,downloaded_at
0,13649,6114,Song,2021-03-16
1,55815,2574,SoundEffect,2021-03-16
2,55815,2562,SoundEffect,2021-03-16
3,57150,3512,SoundEffect,2021-03-16
4,55815,3527,SoundEffect,2021-03-16
...,...,...,...,...
530208,61846,3183,SoundEffect,2022-10-12
530209,118129,6265,Song,2022-10-12
530210,116026,2604,SoundEffect,2022-10-12
530211,116026,2604,SoundEffect,2022-10-12


In [48]:
# 1~40 필요없는 user_id 삭제(이상치 처리)
notid = df_down[df_down['user_id'] <= 40].index
df_down.drop(notid, inplace=True)

### visit 전처리

In [51]:
df_visit = pd.read_csv('data/visit.csv', encoding='utf-8')
df_visit

,id,user_id,browser,os,device_type,source,created_at
0,1,78215,Chrome,Windows,Desktop,NaN,2022-01-17 17:12:07.21728
1,2,71437,Chrome,Windows,Desktop,NaN,2022-01-17 17:12:07.213312
2,3,90407,Chrome,Mac,Desktop,NaN,2022-01-17 17:12:07.225509
3,4,90407,Chrome,Mac,Desktop,NaN,2022-01-17 17:12:07.246342
4,5,78215,Chrome,Windows,Desktop,NaN,2022-01-17 17:12:07.272184
...,...,...,...,...,...,...,...
91125,404700,117634,Chrome,Windows,Desktop,NaN,2022-10-12 16:41:08.2226
91126,404695,116140,Chrome,Windows,Desktop,NaN,2022-10-12 16:39:38.347638
91127,404734,118124,Mobile Safari,iOS,Mobile,NaN,2022-10-12 17:01:11.11918
91128,404745,117777,Chrome,Windows,Desktop,NaN,2022-10-12 17:05:59.423725


In [52]:
df_visit.isnull().sum()

id                 0
user_id            0
browser            3
os                 0
device_type       87
source         89253
created_at         0
dtype: int64

In [53]:
# 결측치 처리
df_visit.fillna('유입경로없음',inplace=True)

In [54]:
# 컬럼명 변경 
df_visit.rename(columns={'created_at':'visted_at'}, inplace=True)


### sfx 전처리

In [55]:
df_sfx = pd.read_csv('data\sfx.csv', encoding='utf-8')
df_sfx

,id,is_free,download_count,play_count,created_at,is_visible,grade,playtime
0,1,True,122,1901,2018-12-21 01:48:19.092798,True,2.3.F,14
1,2,True,34,951,2018-12-21 01:48:19.333004,True,2.3.F,2
2,3,True,27,588,2018-12-21 01:48:19.473285,True,2.3.F,86
3,4,True,40,634,2018-12-21 01:48:19.687445,True,2.3.F,21
4,5,True,34,517,2018-12-21 01:48:19.875785,True,2.3.F,20
...,...,...,...,...,...,...,...,...
3647,4166,False,2,86,2022-09-28 15:57:49.877708,True,2.1.A,6
3648,4167,False,3,59,2022-09-28 15:57:50.996585,True,2.1.A,3
3649,4168,False,35,162,2022-09-28 15:57:51.784597,True,2.1.A,1
3650,4169,False,2,52,2022-09-28 15:57:52.653573,True,2.1.A,9


In [56]:
# 컬럼명 변경
df_sfx.rename(columns={'created_at':'uploaded_at'}, inplace=True)

In [61]:
# datetime으로 형식변경 후 연월일 추출
df_sfx['uploaded_at'] = pd.to_datetime(df_sfx['uploaded_at'])
df_sfx['uploaded_at'] = df_sfx['uploaded_at'].dt.strftime('%Y-%m-%d')

In [58]:
df_sfx.isnull().sum()

id                0
is_free           0
download_count    0
play_count        0
uploaded_at       0
is_visible        0
grade             0
playtime          0
dtype: int64

In [62]:
df_sfx

,id,is_free,download_count,play_count,uploaded_at,is_visible,grade,playtime
0,1,True,122,1901,2018-12-21,True,2.3.F,14
1,2,True,34,951,2018-12-21,True,2.3.F,2
2,3,True,27,588,2018-12-21,True,2.3.F,86
3,4,True,40,634,2018-12-21,True,2.3.F,21
4,5,True,34,517,2018-12-21,True,2.3.F,20
...,...,...,...,...,...,...,...,...
3647,4166,False,2,86,2022-09-28,True,2.1.A,6
3648,4167,False,3,59,2022-09-28,True,2.1.A,3
3649,4168,False,35,162,2022-09-28,True,2.1.A,1
3650,4169,False,2,52,2022-09-28,True,2.1.A,9
